Instalamos las librerias necesarias

In [43]:
!pip install num2words

In [58]:
from google.colab import drive
import pandas as pd
import os
import unicodedata
from num2words import num2words
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import random

Conectamos al drive

In [46]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Seleccionamos las reseñas


In [47]:
datasets_path = 'drive/MyDrive/bases_de_datos'
data_json = 'Cell_Phones_and_Accessories_5.json'
df_cellphones = pd.read_json(os.path.join(datasets_path, data_json), lines=True)
df_cellphones = df_cellphones[~(df_cellphones['reviewText'] == '')].drop_duplicates(subset=['reviewText', 'overall'], keep=False,inplace=False)
reviews = df_cellphones.get('reviewText')
reviews.head()

0    They look good and stick good! I just don't li...
1    These stickers work like the review says they ...
2    These are awesome and make my phone look so st...
3    Item arrived in great time and was in perfect ...
4    awesome! stays on, and looks great. can be use...
Name: reviewText, dtype: object

Diccionario de contracciones

In [49]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

Función para limpiar el texto

In [50]:
def nltk_cleaner(text):
    clean_text = list()
    tokenizer = RegexpTokenizer(r'\w+')
    sw_list = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()

    #Convertir a minúsculas
    text = text.lower().split()

    #Quitar contracciones
    text_aux = []
    for word in text:
        if word in contractions:
            text_aux.append(contractions[word])
        else:
            text_aux.append(word)
    text = " ".join(text_aux)

    # Eliminar acentos, etc
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    # Separar palabras eliminando signos de puntuación
    for word in tokenizer.tokenize(text):
        
        # Eliminar stop words
        if word not in sw_list:
            
        # Eliminar espacios sobrantes y lematizar
            clean_word = lemmatizer.lemmatize(word).strip()
        
        # Convertir dígitos a palabras
            if clean_word.isdigit():
                clean_word = num2words(clean_word, lang='en')

            clean_text.append(clean_word)
            
    return ' '.join(clean_text)

Limpiamos las reseñas

In [53]:
nltk.download('stopwords')
nltk.download('wordnet')
clean_texts = []
for review in reviews:
    clean_texts.append(nltk_cleaner(review))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Seleccionamos una muestra aleatoria

In [60]:
random.sample(clean_texts,5)

['love mercury slim fit flexible case second one buy arrived quickly quality good would definitely recommend buyer five star',
 'rockbed case iphone four four nice hard plastic case like way feel look point using soft rubber otter case one thing like rockbed case quickly pull front pocket jean whereas otter stick inside pocket feel like going pull pocket inside thing care rokbed one come screen protector2 non stick pad back testing non stick pad put iphone dash car five mile straight driving phone fell dash yes rokbed came car mount mounted yet also phone resting hard surface pressing center button typing key left right edge keyboard phone tends rock bit non stick pad three case design make bit difficult press button side may problem long fingernail find difficult adjust volume take picture using side button button top iphone problem yet case coming apart see might happen two small plastic protrusion see picture uploaded approximately zero one height basically keep lower upper part cas

Guardamos las reseñas limpiadas en el dataframe y eliminamos las originales

In [65]:
df_cellphones['cleaned_reviewtext'] = clean_texts
df_cellphones.drop(['reviewText'], axis = 'columns', inplace=True)
df_cellphones.head()

,reviewerID,asin,reviewerName,helpful,overall,summary,unixReviewTime,reviewTime,cleaned_reviewtext
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",4,Looks Good,1400630400,"05 21, 2014",look good stick good like rounded shape always...
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",5,Really great product.,1389657600,"01 14, 2014",sticker work like review say stick great stay ...
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",5,LOVE LOVE LOVE,1403740800,"06 26, 2014",awesome make phone look stylish used one far a...
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",4,Cute!,1382313600,"10 21, 2013",item arrived great time perfect condition howe...
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013",awesome stay look great used multiple apple pr...


Guardamos un dataframe con las columnas de cleaned_reviewtext y overall

In [68]:
df_cellphones_cleaned = df_cellphones.dropna(subset=['cleaned_reviewtext', 'overall'])[['cleaned_reviewtext', 'overall']]
df_cellphones_cleaned.to_csv('drive/MyDrive/bases_de_datos/cleaned_cellphones.csv', index=False)

In [69]:
import numpy as np

print(np.isnan(np.nan))

True
